In [148]:
import numpy as np

In [149]:
def print_array(arr, title):
    """
    prints a 2-D numpy array in a nicer format
    """
    print(title)
    for a in arr:
        for elem in a:
            print("{:.4f}".format(elem).rjust(3), end="\t")
        print(end="\n")

In [150]:
confusion_matrix = np.array([[0.801, 0.304, 0.208, 0.016, 0.032, 0.100],
                             [0.304, 0.778, 0.656, 0.042, 0.131, 0.123],
                             [0.208, 0.656, 0.912, 0.062, 0.094, 0.096],
                             [0.016, 0.042, 0.062, 0.855, 0.154, 0.110],
                             [0.032, 0.131, 0.094, 0.154, 0.958, 0.158],
                             [0.100, 0.123, 0.096, 0.110, 0.158, 0.224]])
node_to_cluster = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
legal = {(1, 1, 1), (1, 0, 0), (0, 1, 0), (0, 0, 1), (0, 0, 0)}

In [151]:
def stochastic_block(node_i, node_j):
    prob = np.random.rand()
    if prob <= confusion_matrix[node_to_cluster[node_i],
                                node_to_cluster[node_j]]:
        return 1
    else:
        return 0


def conditional_block(nodes):
    while True:
        # use stochastic_block model to draw edges
        result = tuple([stochastic_block(nodes[i], nodes[j])
                         for i in range(len(nodes) - 1)
                         for j in range(i + 1, len(nodes))])
        # check the legality of the edges
        if result in legal:
            break
    return result 

In [152]:
from collections import Counter
import pandas as pd
pd.options.display.multi_sparse = False

In [153]:
configs = [
    [1, 1, 1],
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 2],
    [1, 2, 1],
    [2, 1, 1]
]

In [154]:
np.random.seed(42)

tables = []

for _ in range(100):
    table = {}
    for config in configs:
        counter = Counter({legal_config: 0 for legal_config in legal})
        counter.update([conditional_block(config) for _ in range(1000)])
        table[str(tuple(config))] = counter
    
    table_4 = pd.DataFrame(table).sort_index(ascending=False)
    table_4 = table_4 / table_4.sum(axis=0)
    tables.append(table_4.values)

tables = np.array(tables)

In [155]:
tables[0]

array([[0.792, 0.117, 0.12 , 0.1  , 0.584, 0.645, 0.613],
       [0.068, 0.578, 0.053, 0.05 , 0.181, 0.082, 0.081],
       [0.054, 0.082, 0.567, 0.068, 0.104, 0.145, 0.104],
       [0.063, 0.082, 0.078, 0.588, 0.081, 0.077, 0.145],
       [0.023, 0.141, 0.182, 0.194, 0.05 , 0.051, 0.057]])

In [156]:
print_array(tables.mean(0), "mean:")

mean:
0.7891	0.1101	0.1118	0.1102	0.6054	0.6051	0.6053	
0.0648	0.5815	0.0704	0.0715	0.1663	0.0909	0.0904	
0.0636	0.0717	0.5764	0.0716	0.0890	0.1664	0.0909	
0.0641	0.0724	0.0736	0.5803	0.0914	0.0895	0.1664	
0.0184	0.1643	0.1678	0.1664	0.0479	0.0480	0.0470	


In [157]:
print_array(tables.std(0), "sd. dev.")

sd. dev.
0.0134	0.0097	0.0100	0.0094	0.0162	0.0148	0.0153	
0.0081	0.0156	0.0070	0.0084	0.0124	0.0091	0.0082	
0.0071	0.0087	0.0163	0.0090	0.0092	0.0117	0.0091	
0.0080	0.0087	0.0082	0.0170	0.0086	0.0087	0.0118	
0.0043	0.0118	0.0108	0.0117	0.0066	0.0063	0.0076	


In [158]:
print_array(tables.mean(0) + 2 * tables.std(0), "mean + 2 * sd. dev.")
print_array(tables.mean(0) - 2 * tables.std(0), "mean - 2 * sd. dev.")

mean + 2 * sd. dev.
0.8159	0.1296	0.1317	0.1291	0.6377	0.6348	0.6360	
0.0810	0.6126	0.0844	0.0883	0.1911	0.1091	0.1068	
0.0778	0.0890	0.6090	0.0896	0.1075	0.1898	0.1091	
0.0801	0.0899	0.0900	0.6144	0.1087	0.1068	0.1900	
0.0271	0.1879	0.1894	0.1897	0.0612	0.0606	0.0622	
mean - 2 * sd. dev.
0.7622	0.0907	0.0918	0.0913	0.5731	0.5755	0.5746	
0.0487	0.5503	0.0564	0.0547	0.1414	0.0728	0.0739	
0.0494	0.0543	0.5438	0.0537	0.0705	0.1431	0.0727	
0.0481	0.0549	0.0573	0.5462	0.0741	0.0722	0.1428	
0.0098	0.1408	0.1463	0.1431	0.0347	0.0354	0.0318	
